### Оглавление

### Введение
С развитием LLM-моделей спрашивать теоретическую часть или короткие алгоритмы и принципы стало бесполезно, поэтому задание будет одно, но комплексное и приближенное к реальности.

Вы можете использовать любые LLM-модели и Copilot для написания кода, так как в реальной работе сотрудники отдела также имеют доступ ко всем современным инструментам (Sonnet 3.5, GPT-4o, GitHub Copilot и т. п.).

В задании вы будете работать с тестовыми данными по банкоматам и API ЦБ РФ для получения ключевой ставки.

---

### Специфика данных fees
`fees` — тарифы за обслуживание банкомата инкассаторами. Так как банкоматы могут находиться в разных регионах, тарифы у них могут отличаться.

- **CashDeliveryFixedFee** — фиксированная стоимость доставки наличных (не зависит от суммы).
- **CashDeliveryPercentageFee** — процент от суммы доставленных наличных. Он добавляется к фиксированной стоимости. Если в таблице указано `0.0001`, это значит 0.01% от суммы.
- **CashDeliveryMinFee** — минимальная сумма, которую с нас возьмут по `CashDeliveryPercentageFee`. Например, если мы попросим инкассаторов в ATM_4 довезти всего 10 000 рублей, то с нас возьмут 5250 рублей (фиксированная стоимость), а так как `10 000 * 0.04% < 450 рублей`, то дополнительно возьмут ещё 450 рублей. Итог: 5250 + 450 рублей.

**CashCollection** — это пример сложного тарифа, который учитывает специфику работы с банкоматом. В банкомат деньги не докладывают, а меняют сразу кассету. Есть провайдеры, которые тарифицируют не только доставку, но и пересчёт денег в извлечённой кассете. Например, в ATM_4 у нас именно такой тариф.  
Пример: у нас в банкомате осталось 500 тыс. рублей, а мы хотим, чтобы у него был баланс 2 млн. Тогда мы заказываем довезти кассету на 2 млн. и платим по тарифу за доставку. При доставке старая кассета извлекается, и мы также по тарифу платим за пересчёт 500 тыс. рублей (0.45%, но не менее 1140 рублей).

- **CashCollectionFixedFee** — фиксированная стоимость за извлечение старой кассеты (в данных примерах нулевая).
- **CashCollectionPercentageFee** — процент от суммы извлечённой кассеты.
- **CashCollectionMinFee** — минимальная сумма, которую с нас возьмут по `CashCollectionPercentageFee`.

---

### Специфика данных transactions
Таблица содержит данные по снятиям, пополнениям (инкассациям) и балансу банкомата на конец дня.  
Считаем, что банкоматы в начале года пустые и не работали, поэтому баланс на конец дня равен 0, пока не случится первая инкассация.

- **bal_end_of_day** — баланс на конец дня.
- **cash_in** — пополнение в результате инкассации.
- **cash_out** — снятие наличных клиентами.

---

In [47]:
import pandas as pd
import requests

from datetime import datetime
from lxml import etree
import xml.etree.ElementTree as ET

In [3]:
# Забираем данные по тарфиам на обслуживание банкоматов
fees = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/fees.parquet', engine='pyarrow')
# статистика операция по дням.
transactions = pd.read_parquet('https://storage.yandexcloud.net/norvpublic/transactions.parquet', engine='pyarrow')

In [14]:
fees

,ATM_ID,CashDeliveryFixedFee,CashDeliveryPercentageFee,CashDeliveryMinFee,CashCollectionFixedFee,CashCollectionPercentageFee,CashCollectionMinFee
0,ATM_1,1365.0,0.0001,NaN,NaN,NaN,NaN
1,ATM_2,1365.0,0.0001,NaN,NaN,NaN,NaN
2,ATM_4,5250.0,0.0004,450.0,0.0,0.0045,1140.0
3,ATM_3,2250.0,0.0001,675.0,NaN,NaN,NaN


In [5]:
fees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ATM_ID                       4 non-null      string 
 1   CashDeliveryFixedFee         4 non-null      float64
 2   CashDeliveryPercentageFee    4 non-null      float64
 3   CashDeliveryMinFee           2 non-null      float64
 4   CashCollectionFixedFee       1 non-null      float64
 5   CashCollectionPercentageFee  1 non-null      float64
 6   CashCollectionMinFee         1 non-null      float64
dtypes: float64(6), string(1)
memory usage: 356.0 bytes


In [6]:
transactions

,date,ATM_ID,bal_end_of_day,cash_in,cash_out
0,2024-01-12 00:00:00+00:00,ATM_1,0.0,0.0,0.0
1,2024-01-12 00:00:00+00:00,ATM_2,0.0,0.0,0.0
2,2024-01-12 00:00:00+00:00,ATM_3,0.0,0.0,0.0
3,2024-01-12 00:00:00+00:00,ATM_4,0.0,0.0,0.0
4,2024-01-13 00:00:00+00:00,ATM_1,0.0,0.0,0.0
...,...,...,...,...,...
1171,2024-10-30 00:00:00+00:00,ATM_4,747650.0,0.0,1000.0
1172,2024-10-31 00:00:00+00:00,ATM_1,3215500.0,0.0,171600.0
1173,2024-10-31 00:00:00+00:00,ATM_2,4754600.0,0.0,395100.0
1174,2024-10-31 00:00:00+00:00,ATM_3,2277000.0,0.0,1000.0


In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype                  
---  ------          --------------  -----                  
 0   date            1176 non-null   datetime64[ms, Etc/UTC]
 1   ATM_ID          1176 non-null   string                 
 2   bal_end_of_day  1176 non-null   float64                
 3   cash_in         1176 non-null   float64                
 4   cash_out        1176 non-null   float64                
dtypes: datetime64[ms, Etc/UTC](1), float64(3), string(1)
memory usage: 46.1 KB


### Часть 1 — упущенный процентный доход
<a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

Специалисту по ML важно уметь получать данные с различных API и читать документацию. Для расчёта упущенного процентного дохода нужно обратиться к API ЦБ РФ и получить динамику ключевой ставки за 2024 год.

https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?op=KeyRate

Для запроса к API не нужен токен и регистрация. ЦБ РФ для части данных предпочитает использовать SOAP.

Учитывая, что хранить наличные деньги в банкомате — не самое удачное инвестиционное решение, посчитайте упущенный процентный доход для каждого банкомата. Рассчитываем, что банк мог бы вложить эти деньги и получить доход, равный ключевой ставке ЦБ РФ, актуальной на день баланса банкомата. Добавьте к таблице `transactions` столбец с упущенной процентной выгодой.


In [60]:
url = "https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx"
headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "http://web.cbr.ru/KeyRate"
}
soap_request = '''<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
               xmlns:xsd="http://www.w3.org/2001/XMLSchema"
               xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <KeyRate xmlns="http://web.cbr.ru/">
      <fromDate>2024-01-01</fromDate>
      <ToDate>2024-12-31</ToDate>
    </KeyRate>
  </soap:Body>
</soap:Envelope>'''

In [61]:
response = requests.post(url, data=soap_request, headers=header)
if response.status_code != 200:
    print(f"Ошибка запроса: {response.status_code}")
else:
    print(response.text)

<?xml version="1.0" encoding="utf-8"?><soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema"><soap:Body><KeyRateResponse xmlns="http://web.cbr.ru/"><KeyRateResult><xs:schema id="KeyRate" xmlns="" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:msdata="urn:schemas-microsoft-com:xml-msdata"><xs:element name="KeyRate" msdata:IsDataSet="true" msdata:UseCurrentLocale="true"><xs:complexType><xs:choice minOccurs="0" maxOccurs="unbounded"><xs:element name="KR"><xs:complexType><xs:sequence><xs:element name="DT" msdata:Caption="Дата" type="xs:dateTime" minOccurs="0" /><xs:element name="Rate" msdata:Caption="Ставка %" type="xs:decimal" minOccurs="0" /></xs:sequence></xs:complexType></xs:element></xs:choice></xs:complexType></xs:element></xs:schema><diffgr:diffgram xmlns:msdata="urn:schemas-microsoft-com:xml-msdata" xmlns:diffgr="urn:schemas-microsoft-com:xml-diffgram-v1"><KeyRate xm

In [70]:
if response.status_code == 200:
    # Сохранение ответа в файл
    with open("response.xml", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("XML-ответ сохранен в файл 'response.xml'")
else:
    print("Error:", response.status_code)

XML-ответ сохранен в файл 'response.xml'


In [74]:
# Перебор структуры XML
tree = ET.ElementTree(ET.fromstring(response.text))
root = tree.getroot()

In [77]:
namespaces = {
    'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
    'web': 'http://web.cbr.ru/',
    'xsd': 'http://www.w3.org/2001/XMLSchema'
}

# Перебираем все элементы в корне
for elem in root.iter():
    print(f"Тег: {elem.tag}, Атрибуты: {elem.attrib}, Значение: {elem.text}")

# Например, найти элемент KeyRateResponse
key_rate_response = root.find('.//web:KeyRateResponse', namespaces)
if key_rate_response is not None:
    print("\nKeyRateResponse найден:")
    for child in key_rate_response:
        print(f"Тег: {child.tag}, Значение: {child.text}")

# Пример извлечения значений
key_rate_result = root.find('.//web:KeyRateResult', namespaces)
if key_rate_result is not None:
    rate = key_rate_result.find('web:rate', namespaces)
    if rate is not None:
        print("\nRate:", rate.text)

    currency = key_rate_result.find('web:currency', namespaces)
    if currency is not None:
        print("Currency:", currency.text)

Тег: {http://schemas.xmlsoap.org/soap/envelope/}Envelope, Атрибуты: {}, Значение: None
Тег: {http://schemas.xmlsoap.org/soap/envelope/}Body, Атрибуты: {}, Значение: None
Тег: {http://web.cbr.ru/}KeyRateResponse, Атрибуты: {}, Значение: None
Тег: {http://web.cbr.ru/}KeyRateResult, Атрибуты: {}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}schema, Атрибуты: {'id': 'KeyRate'}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}element, Атрибуты: {'name': 'KeyRate', '{urn:schemas-microsoft-com:xml-msdata}IsDataSet': 'true', '{urn:schemas-microsoft-com:xml-msdata}UseCurrentLocale': 'true'}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}complexType, Атрибуты: {}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}choice, Атрибуты: {'minOccurs': '0', 'maxOccurs': 'unbounded'}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}element, Атрибуты: {'name': 'KR'}, Значение: None
Тег: {http://www.w3.org/2001/XMLSchema}complexType, Атрибуты: {}, Значение: None
Тег: {http:

### Часть 2 - расходы на инкассацию

Теперь к данным по банкоматам, кроме упущенного процентного дохода, необходимо добавить столбец расходов на инкассацию. Для простоты считаем, что мы не платим за аренду помещения и страхование, поэтому нам достаточно данных по тарифам на инкассацию (`fees`).

### Часть 3 - анализ данных

Посмотрите на данные о снятиях, инкассациях и балансе. В данных подобраны банкоматы, отличающиеся по характеру использования и частоте инкассаций.  
(Эту операцию вы делаете для себя, чтобы лучше понять специфику.)

### Часть 4 - меняем бизнес с помощью ML

Заключительная и самая интересная часть. К данному этапу у нас уже рассчитан упущенный процентный доход и расходы на инкассацию. Подумайте, как использовать ваши знания, чтобы оптимизировать процесс инкассации и уменьшить издержки.

Для данной задачи считаем, что нас устроит Service Level на уровне 90%. Это значит, что если в 9 из 10 случаев наши клиенты получают нужные им суммы, нас это устраивает.

Напоминаем, что технически в банкомат нельзя довнести сумму, и кассета меняется полностью. Соответственно, с нас берут оплату за полную кассету (если вдруг решите подойти к задаче через классическую формулу EOQ).

Вы можете подойти к задаче абсолютно любым способом и использовать все возможности ООП, Python и любых библиотек. Всё как в реальной работе, где вас никто не ограничивает.

Эффективность своего решения вы можете показать с помощью ретротестирования. Считаем, что каждая кассета может вмещать абсолютно любые суммы. Чтобы добавить реализма, вы можете считать, что деньги нужно заказывать за 3 дня до их доставки.


### Заключение

Решение вы можете предоставить любым способом: репозиторий GitHub, Google Colab, конвертированный IPython Notebook в PDF и т. п.

По срокам вас не ограничивают, но учитывайте, что в это же время задачу могут решать и другие кандидаты. Таким образом, вы участвуете в конкурсе с ними.